# Dependencies

In [ ]:
# !py -m pip install pyvista --upgrade
# !py -m pip install ipympl --upgrade
# !py -m pip install trame --upgrade
# !py -m pip install scipy --upgrade
# !py -m pip install shapely --upgrade
# !py -m pip install tqdm - upgrade
# !py -m pip install tabulate --upgrade
# !py -m pip install colorama

# Imports

In [1]:
import pyvista as pv
import numpy as np
import os

import math
from math import sqrt, cos, sin, pi, atan2, degrees, floor, acos

import matplotlib.pyplot as plt
import imageio

from scipy.spatial import ConvexHull, convex_hull_plot_2d
from scipy import optimize
from scipy.special import comb
from scipy.interpolate import interp1d

import shapely as sp
from shapely import affinity, distance
from shapely.geometry import Polygon, Point, LinearRing, LineString, MultiPoint
from shapely.plotting import plot_polygon, plot_points, plot_line
from shapely.ops import transform, nearest_points, split, polygonize
from shapely.affinity import rotate

import shutil
from tabulate import tabulate
from colorama import init, Fore, Back, Style
import itertools
from tqdm.notebook import tqdm

# Functions

In [56]:
def matan(point):
    angle = atan2(point.y, point.x)
    if angle <= 0: angle += 2*pi
    return angle

def norma(point):
    return Point(0,0).distance(point)

def Swap(val1, val2):
    val1 = val1 + val2
    val2 = val1 - val2
    val1 = val1 - val2
    return val1, val2

def GetContourFromOBJ(filename):
    return LinearRing(MultiPoint(np.delete(pv.read(filename).project_points_to_plane().points, 2, 1)).convex_hull.boundary.coords)

def GetCornerIndices(pts):
    al = []
    for i in range(len(pts)):
        al.append([pts[i].x, pts[i].y])

    a = np.array(al)
    dx_dt = np.gradient(a[:, 0])
    dy_dt = np.gradient(a[:, 1])
    velocity = np.array([ [dx_dt[i], dy_dt[i]] for i in range(dx_dt.size)])

    ds_dt = np.sqrt(dx_dt * dx_dt + dy_dt * dy_dt)

    tangent = np.array([1/ds_dt]).transpose() * velocity

    tangent_x = tangent[:, 0]
    tangent_y = tangent[:, 1]

    deriv_tangent_x = np.gradient(tangent_x)
    deriv_tangent_y = np.gradient(tangent_y)

    dT_dt = np.array([ [deriv_tangent_x[i], deriv_tangent_y[i]] for i in range(deriv_tangent_x.size)])

    length_dT_dt = np.sqrt(deriv_tangent_x * deriv_tangent_x + deriv_tangent_y * deriv_tangent_y)

    normal = np.array([1/length_dT_dt]).transpose() * dT_dt

    d2s_dt2 = np.gradient(ds_dt)
    d2x_dt2 = np.gradient(dx_dt)
    d2y_dt2 = np.gradient(dy_dt)

    curvature = np.abs(d2x_dt2 * dy_dt - dx_dt * d2y_dt2) / (dx_dt * dx_dt + dy_dt * dy_dt)**1.5
    t_component = np.array([d2s_dt2] * 2).transpose()
    n_component = np.array([curvature * ds_dt * ds_dt] * 2).transpose()

    acceleration = t_component * tangent + n_component * normal

    # plt.plot(curvature)
    return [idx for idx, val in enumerate(curvature) if val>=np.max(curvature)/2]

def ShiftConcatenate(data, shift):
    data = data.copy()
    for _ in range(shift):
        data.append(data.pop(0))
    return data

def GetSmoothedPear(pear):
    pear_original_pts, pts = [Point(pear.coords[i][0], pear.coords[i][1]) for i in range(len(pear.coords))], []
    for pt in pear_original_pts: pts.append([pt.x, pt.y])
    pts = sorted(pts, key=ClockwiseAngleAndDistance)
    pear_original__pts = [Point(pt[0], pt[1]) for pt in pts]
    corner_indices = GetCornerIndices(pear_original_pts)
    if len(corner_indices) > 0: 
        pear_original_shifted_pts = ShiftConcatenate(pear_original_pts, corner_indices[0])

    pear_original_shifted_pts, pts = list(set(pear_original_shifted_pts)), []
    for pt in pear_original_shifted_pts: pts.append([pt.x, pt.y])
    pts = sorted(pts, key=ClockwiseAngleAndDistance)
    pear_original_shifted_pts = [Point(pt[0], pt[1]) for pt in pts]

    corner_indices = GetCornerIndices(pear_original_shifted_pts)
    if len(corner_indices) > 0: 
        pear_original_shifted_pts = ShiftConcatenate(pear_original_shifted_pts, corner_indices[0])

    x, y = [pt.x for pt in pear_original_shifted_pts[1:]], [pt.y for pt in pear_original_shifted_pts[1:]]

    points = np.array([x, y]).T
    distance = np.cumsum( np.sqrt(np.sum( np.diff(points, axis=0)**2, axis=1 )) )
    distance = np.insert(distance, 0, 0)/distance[-1]
    interpolator =  interp1d(distance, points, kind='cubic', axis=0)
    interpolated_points = interpolator(np.linspace(0, 1, 1000))
    pts = [Point(pt[0], pt[1]) for pt in interpolated_points]
    pts.append(pear_original_shifted_pts[0])

    return LinearRing(pts)

def GetBaloonedContour(contour, baloon_distance):
    return LinearRing(LineString(contour).offset_curve(baloon_distance, quad_segs=int(2**5), join_style=2, mitre_limit=5.0))

def ClockwiseAngleAndDistance(point):
    origin, refvec = [0,0], [1,0]
    # Vector between point and the origin: v = p - o
    vector = [point[0]-origin[0], point[1]-origin[1]]
    # Length of vector: ||v||
    lenvector = math.hypot(vector[0], vector[1])
    # If length is zero there is no angle
    if lenvector == 0:
        return -math.pi, 0
    # Normalize vector: v/||v||
    normalized = [vector[0]/lenvector, vector[1]/lenvector]
    dotprod  = normalized[0]*refvec[0] + normalized[1]*refvec[1]     # x1*x2 + y1*y2
    diffprod = refvec[1]*normalized[0] - refvec[0]*normalized[1]     # x1*y2 - y1*x2
    angle = math.atan2(diffprod, dotprod)
    # Negative angles represent counter-clockwise angles so we need to subtract them 
    # from 2*pi (360 degrees)
    if angle < 0:
        return 2*math.pi+angle, lenvector
    # Return first the angle because that's the primary sorting criterium
    # but if two vectors have the same angle then the shorter distance should come first.
    return angle, lenvector

def GetOrderedIntersectionPoints(geom1, geom2):
    intersection, pts, spts = geom1.intersection(geom2), [], []
    if intersection.geom_type == 'LineString':
        return [], []
    else:
        for geom in intersection.geoms: pts.append([geom.x, geom.y])
        pts = sorted(pts, key=ClockwiseAngleAndDistance)
        for pt in reversed(pts): spts.append(Point(pt[0], pt[1]))
        angle1, angle2 = matan(spts[0]), matan(spts[1])
        line = LineString([(0,0), (30*cos((angle1+angle2)/2), 30*sin((angle1+angle2)/2))])
        pt1, pt2 = geom1.intersection(line), geom2.intersection(line)
        if norma(pt1) < norma(pt2): spts.append(spts.pop(0))
        return spts, [matan(pt) for pt in spts]

def GetSimpleOrderedIntersectionPoints(geom1, geom2):
    intersection, pts, spts = geom1.intersection(geom2), [], []
    if intersection.geom_type == 'LineString':
        return [], []
    elif intersection.geom_type == 'Point':
        return [intersection], [matan(intersection)]
    else:
        for geom in intersection.geoms: pts.append([geom.x, geom.y])
        pts = sorted(pts, key=ClockwiseAngleAndDistance)
        for pt in reversed(pts): spts.append(Point(pt[0], pt[1]))
        return spts, [matan(pt) for pt in spts]

def Contour2Points(contour, begin_angle, n):
    contour_pts = []
    for i in range(n):
        angle = begin_angle + (i/n)*2*pi
        intersection = contour.intersection(LineString([(0,0), (30*cos(angle), 30*sin(angle))]))
        if intersection.geom_type == 'Point': contour_pts.append(intersection)
        else: contour_pts.append(intersection.geoms[-1])
    return contour_pts

def GetNormalVectors(pts):
    al = []
    for i in range(len(pts)):
        al.append([pts[i].x, pts[i].y])

    a = np.array(al)
    dx_dt = np.gradient(a[:, 0])
    dy_dt = np.gradient(a[:, 1])
    velocity = np.array([ [dx_dt[i], dy_dt[i]] for i in range(dx_dt.size)])

    ds_dt = np.sqrt(dx_dt * dx_dt + dy_dt * dy_dt)

    tangent = np.array([1/ds_dt]).transpose() * velocity

    tangent_x = tangent[:, 0]
    tangent_y = tangent[:, 1]

    deriv_tangent_x = np.gradient(tangent_x)
    deriv_tangent_y = np.gradient(tangent_y)

    dT_dt = np.array([ [deriv_tangent_x[i], deriv_tangent_y[i]] for i in range(deriv_tangent_x.size)])

    length_dT_dt = np.sqrt(deriv_tangent_x * deriv_tangent_x + deriv_tangent_y * deriv_tangent_y)

    normal = np.array([1/length_dT_dt]).transpose() * dT_dt
    return [Point(-pt[0]/sqrt(pt[0]**2+pt[1]**2), -pt[1]/sqrt(pt[0]**2+pt[1]**2)) for pt in normal]

def GetRotatedReperIndices(angles, intersection_angles):
    indices = []
    local_angles = [angle-intersection_angles[0] for angle in intersection_angles]
    # indices.append(0)
    for angle in local_angles:
        for i in range(len(angles)):
            if angles[i] > angle:
                indices.append(i)
                break
    # indices.pop(-1)
    return indices, len(indices)

def GetReperIndices(angles, intersection_angles):
    indices = []
    for angle in intersection_angles:
        for i in range(len(angles)):
            if angles[i] > angle:
                indices.append(i)
                break
    return indices, len(indices)

def GetNormals(pts, blpts, n):
    normals = []
    for i in range(n):
        x, y = blpts[i].x-pts[i].x, blpts[i].y-pts[i].y
        norm = sqrt(x**2+y**2)
        assert abs(norm) > 1e-14
        normals.append(Point(x/norm, y/norm))
    return normals

def GetBaloonedContourWithContainPercentage(geom1, geom2, n, contain_percent):
    geom1_pts, geom2_pts = Contour2Points(geom1, 0, n),  Contour2Points(geom2, 0, n)
    dsts = [geom1_pts[i].distance(geom2_pts[i]) for i in range(n)]
    hist, bin_edges = np.histogram(dsts, bins=floor(n/4))
    # plt.hist(dsts, bins=floor(n/4)); plt.show()
    sum, percent, whole_sum, i_edge = 0, 0, hist.sum(), 0
    for i in range(len(hist)):
        sum += hist[i]
        percent = sum / whole_sum
        if percent > contain_percent:
            i_edge = i-1
            break
    blgeom2 = GetBaloonedContour(geom2, np.min(dsts)+(i_edge/len(hist))*(np.max(dsts)-np.min(dsts)))
    spts, intersection_angles = GetOrderedIntersectionPoints(geom1, blgeom2)
    return blgeom2, spts

def GetMaxDistance(geom1_pts, geom2):
    dst_max = 0.0
    for i in range(len(geom1_pts)):
        pt1, pt2 = nearest_points(geom2, geom1_pts[i])
        dst1, dst2 = geom1_pts[i].distance(pt1), geom1_pts[i].distance(pt2)
        dst = max(dst1, dst2)
        if dst > dst_max: dst_max = dst
    return dst_max

def GetTangentCycle(geom1, geom2, disk_delta):
    intersection, bldistance = MultiPoint(), 0
    while intersection.geom_type == "MultiPoint":
        bldistance += disk_delta
        blgeom2 = LineString(geom2).offset_curve(-bldistance, join_style=2)
        intersection = geom1.intersection(blgeom2)
    return blgeom2, bldistance
    
def GetNextCycleContour(geom1, geom2, dst_delta, disk_delta):
    blgeom2, dst = LineString(geom2).offset_curve(-dst_delta, join_style=2), dst_delta
    spts, val = GetSimpleOrderedIntersectionPoints(geom1, blgeom2)
    # if len(spts) == 0:
    #     blgeom2, dst = GetTangentCycle(geom1, geom2, disk_delta)
    #     spts, val = GetSimpleOrderedIntersectionPoints(geom1, blgeom2)
    return blgeom2, spts, dst

def GetPointOnLineWithDistance(line, distance):
    if distance < 0.0 or distance > line.length:
        return None
    coords = list(line.coords)
    for i, p in enumerate(coords):
        pd = line.project(Point(p))
        if pd == distance:
            return Point(coords[i][0], coords[i][1])
        if pd > distance:
           return line.interpolate(distance)

def GetCurrentShape(current_rough_pts, disk, angles, n):
    diamond_ring = LinearRing(current_rough_pts)
    if disk.intersects(diamond_ring):
        rough = sp.difference(Polygon(current_rough_pts), disk).exterior
        npts = diamond_ring.intersection(disk)
        if npts.geom_type == 'LineString':
            angle_edge1 = atan2(npts.coords[0][1], npts.coords[0][0])
            # if angle_edge1 < 0: angle_edge1 += 2*pi
            angle_edge2 = atan2(npts.coords[-1][1], npts.coords[-1][0])
            # if angle_edge2 < 0: angle_edge2 += 2*pi
        else:
            if npts.geom_type == 'Point':
                return current_rough_pts
            else:
                angle_edge1 = atan2(npts.geoms[-1].coords[0][1], npts.geoms[-1].coords[0][0])
                # if angle_edge1 < 0: angle_edge1 += 2*pi
                angle_edge2 = atan2(npts.geoms[0].coords[-1][1], npts.geoms[0].coords[-1][0])
                # if angle_edge2 < 0: angle_edge2 += 2*pi
        
        if angle_edge1 > angle_edge2: angle_edge2 += 2*pi
        for i in range(n):
            # cur_angle = angles[i]-rotation_angle
            cur_angle = atan2(current_rough_pts[i].y, current_rough_pts[i].x)
            if angle_edge1 <= cur_angle and cur_angle <= angle_edge2:
                line = LineString([(0,0), (30*cos(cur_angle), 30*sin(cur_angle))])
                current_rough_pts[i] = rough.intersection(line)
    return current_rough_pts

def GetNextBrutingCycle(trace1_pts, trace2_pts, dst):
    assert len(trace1_pts) == len(trace2_pts)
    trace_pts = []
    for i in range(len(trace1_pts)):
        x1, y1 = trace1_pts[i].x, trace1_pts[i].y
        x2, y2 = trace2_pts[i].x, trace2_pts[i].y
        nrm = sqrt((x2-x1)**2+(y2-y1)**2)
        assert abs(nrm) > 1e-14
        trace_pts.append(Point(x1+dst*(x2-x1)/nrm, y1+dst*(y2-y1)/nrm))
    return trace_pts

def TestTrace(time, rangles, xvals, dvals, j, q, print_flag, n,
              min_vel_rangle, min_acc_rangle, max_vel_rangle, max_acc_rangle,
              min_xval, min_vel_xval, min_acc_xval, max_xval, max_vel_xval, max_acc_xval,
              min_dval, min_vel_dval, min_acc_dval, max_dval, max_vel_dval, max_acc_dval):
    drangles = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, rangles[1:], rangles)]
    dxvals = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, xvals[1:], xvals)]
    ddvals = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, dvals[1:], dvals)]
    ddrangles = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, drangles[1:], drangles)]
    ddxvals = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, dxvals[1:], dxvals)]
    dddvals = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, ddvals[1:], ddvals)]
    test = [Style.BRIGHT+Back.MAGENTA+str(j+1)+'-'+str(q+1)+Back.RESET+Style.NORMAL]
    if len(time) > n: test.append(Style.BRIGHT+Back.RED+'failure'+Back.RESET+Style.NORMAL+' ('+str(len(time))+')')
    else: test.append(Style.BRIGHT+Back.GREEN+'success'+Back.RESET+Style.NORMAL+' ('+str(len(time))+')')
    idx = np.where((max_vel_rangle<np.array(drangles))|(np.array(drangles)<min_vel_rangle))[0]
    if idx.size > 0:
        test.append(Style.BRIGHT+Back.RED+'failure'+Back.RESET+Style.NORMAL+' ('+str(idx.size)+')')
        if print_flag == True:
            print('trespassing rotation velocity limits in cycle № '+str(j)+'-'+str(q))
            print('\t[min_limit,max_limit] = ['+str(min_vel_rangle)+','+str(max_vel_rangle)+']\n\t\ttime\tvalue')
            for id in idx: print('\t\t'+str(time[id])+'\t'+str(np.array(drangles[id])))
    else: test.append(Style.BRIGHT+Back.GREEN+'success'+Back.RESET+Style.NORMAL+' (0)')
    idx = np.where((max_acc_rangle<np.array(ddrangles))|(np.array(ddrangles)<min_acc_rangle))[0]
    if idx.size > 0:
        test.append(Style.BRIGHT+Back.RED+'failure'+Back.RESET+Style.NORMAL+' ('+str(idx.size)+')')
        if print_flag == True:
            print('trespassing rotation acceleration limits in cycle № '+str(j)+'-'+str(q))
            print('\t[min_limit,max_limit] = ['+str(min_acc_rangle)+','+str(max_acc_rangle)+']\n\t\ttime\t\t\tvalue')
            for id in idx: print('\t\t'+str(time[id])+'\t'+str(np.array(ddrangles[id])))
    else: test.append(Style.BRIGHT+Back.GREEN+'success'+Back.RESET+Style.NORMAL+' (0)')
    idx = np.where((max_xval<np.array(xvals))|(np.array(xvals)<min_xval))[0]
    if idx.size > 0:
        test.append(Style.BRIGHT+Back.RED+'failure'+Back.RESET+Style.NORMAL+' ('+str(idx.size)+')')
        if print_flag == True:
            print('trespassing diamond coordinate limits in cycle № '+str(j)+'-'+str(q))
            print('\t[min_limit,max_limit] = ['+str(min_xval)+','+str(max_xval)+']\n\t\ttime\t\t\tvalue')
            for id in idx: print('\t\t'+str(time[id])+'\t'+str(np.array(xvals[id])))
    else: test.append(Style.BRIGHT+Back.GREEN+'success'+Back.RESET+Style.NORMAL+' (0)')
    idx = np.where((max_vel_xval<np.array(dxvals))|(np.array(dxvals)<min_vel_xval))[0]
    if idx.size > 0:
        test.append(Style.BRIGHT+Back.RED+'failure'+Back.RESET+Style.NORMAL+' ('+str(idx.size)+')')
        if print_flag == True:
            print('trespassing diamond velocity limits in cycle № '+str(j)+'-'+str(q))
            print('\t[min_limit,max_limit] = ['+str(min_vel_xval)+','+str(max_vel_xval)+']\n\t\ttime\tvalue')
            for id in idx: print('\t\t'+str(time[id])+'\t'+str(np.array(dxvals[id])))   
    else: test.append(Style.BRIGHT+Back.GREEN+'success'+Back.RESET+Style.NORMAL+' (0)')
    idx = np.where((max_acc_xval<np.array(ddxvals))|(np.array(ddxvals)<min_acc_xval))[0]
    if idx.size > 0:
        test.append(Style.BRIGHT+Back.RED+'failure'+Back.RESET+Style.NORMAL+' ('+str(idx.size)+')')
        if print_flag == True:
            print('trespassing diamond acceleration limits in cycle № '+str(j)+'-'+str(q))
            print('\t[min_limit,max_limit] = ['+str(min_acc_xval)+','+str(max_acc_xval)+']\n\t\ttime\t\t\tvalue')
            for id in idx: print('\t\t'+str(time[id])+'\t'+str(np.array(ddxvals[id])))
    else: test.append(Style.BRIGHT+Back.GREEN+'success'+Back.RESET+Style.NORMAL+' (0)')
    idx = np.where((max_dval<np.array(dvals))|(np.array(dvals)<min_dval))[0]
    if idx.size > 0:
        test.append(Style.BRIGHT+Back.RED+'failure'+Back.RESET+Style.NORMAL+' ('+str(idx.size)+')')
        if print_flag == True:
            print('trespassing disk limits in cycle № '+str(j)+'-'+str(q))
            print('\t[min_limit,max_limit] = ['+str(min_dval)+','+str(max_dval)+']\n\t\ttime\t\t\tvalue')
            for id in idx: print('\t\t'+str(time[id])+'\t'+str(np.array(dvals[id])))
    else: test.append(Style.BRIGHT+Back.GREEN+'success'+Back.RESET+Style.NORMAL+' (0)')
    idx = np.where((max_vel_dval<np.array(ddvals))|(np.array(ddvals)<min_vel_dval))[0]
    if idx.size > 0:
        test.append(Style.BRIGHT+Back.RED+'failure'+Back.RESET+Style.NORMAL+' ('+str(idx.size)+')')
        if print_flag == True:
            print('trespassing disk velocity limits in cycle № '+str(j)+'-'+str(q))
            print('\t[min_limit,max_limit] = ['+str(min_vel_dval)+','+str(max_vel_dval)+']\n\t\ttime\tvalue')
            for id in idx: print('\t\t'+str(time[id])+'\t'+str(np.array(ddvals[id])))
    else: test.append(Style.BRIGHT+Back.GREEN+'success'+Back.RESET+Style.NORMAL+' (0)')
    idx = np.where((max_acc_dval<np.array(dddvals))|(np.array(dddvals)<min_acc_dval))[0]
    if idx.size > 0:
        test.append(Style.BRIGHT+Back.RED+'failure'+Back.RESET+Style.NORMAL+' ('+str(idx.size)+')')
        if print_flag == True:
            print('trespassing disk acceleration limits in cycle № '+str(j)+'-'+str(q))
            print('\t[min_limit,max_limit] = ['+str(min_acc_dval)+','+str(max_acc_dval)+']\n\t\ttime\t\t\tvalue')
            for id in idx: print('\t\t'+str(time[id])+'\t'+str(np.array(dddvals[id])))
    else: test.append(Style.BRIGHT+Back.GREEN+'success'+Back.RESET+Style.NORMAL+' (0)')
    return test

def GraphTrace(time, rangles, xvals, dvals, R, path, j, q,
              min_vel_rangle, min_acc_rangle, max_vel_rangle, max_acc_rangle,
              min_xval, min_vel_xval, min_acc_xval, max_xval, max_vel_xval, max_acc_xval,
              min_dval, min_vel_dval, min_acc_dval, max_dval, max_vel_dval, max_acc_dval):
    drangles = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, rangles[1:], rangles)]
    dxvals = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, xvals[1:], xvals)]
    ddvals = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, dvals[1:], dvals)]

    ddrangles = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, drangles[1:], drangles)]
    ddxvals = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, dxvals[1:], dxvals)]
    dddvals = [(y2-y0)/(x2-x0) for x2, x0, y2, y0 in zip(time[1:], time, ddvals[1:], ddvals)]

    colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
    fig, axs = plt.subplots(3, 3)
    fig.set_size_inches(24.0, 12.0)
    fig.suptitle('Graph of the main values of the bruting\nB4969-B_pear, disk radius ' + str(round(float(R), 2)) + ' mm\nswing zone №' + str(j+1) + ', cycle №' + str(q+1), fontsize=14, fontweight='bold')

    plt.figtext(0.07, 0.76, 'VALUE', fontsize=14)
    plt.figtext(0.05, 0.49, 'VELOCITY', fontsize=14)
    plt.figtext(0.025, 0.22, 'ACCELERATION', fontsize=14)

    axs[0,0].plot(time, rangles, marker='.', color="red", markersize=0.005)
    axs[0,0].set_ylabel('rotation angle of diamond')
    axs[0,0].set_title('ROTATION ANGLE')
    axs[0,0].grid(True)

    axs[1,0].plot(time[:-1], drangles, marker='.', color="red", markersize=0.1)
    axs[1,0].plot(time[:-1], [min_vel_rangle]*(len(time)-1), marker='.', color="blue", markersize=0.1)
    axs[1,0].plot(time[:-1], [max_vel_rangle]*(len(time)-1), marker='.', color="blue", markersize=0.1)
    axs[1,0].set_ylabel('rotation velocity of diamond')
    # axs[1].set_ylim([-2000, 2000])
    axs[1,0].grid(True)

    axs[2,0].plot(time[:-2], ddrangles, marker='.', color="red", markersize=0.1)
    axs[2,0].plot(time[:-2], [min_acc_rangle]*(len(time)-2), marker='.', color="blue", markersize=0.1)
    axs[2,0].plot(time[:-2], [max_acc_rangle]*(len(time)-2), marker='.', color="blue", markersize=0.1)
    axs[2,0].set_ylabel('rotation acceleration of diamond')
    # axs[2,0].set_ylim([-2000, 2000])
    axs[2,0].set_xlabel('time')
    axs[2,0].grid(True)

    axs[0,1].plot(time, xvals, marker='.', linestyle="-", color="red", markersize=0.1)
    # axs[0,1].plot(time[j][q], [min_xval]*(len(time[j][q])), marker='.', color="blue", markersize=0.1)
    # axs[0,1].plot(time[j][q], [max_xval]*(len(time[j][q])), marker='.', color="blue", markersize=0.1)
    axs[0,1].set_ylabel('ordinate of diamond center')
    axs[0,1].set_title('DIAMOND CENTER')
    axs[0,1].grid(True)

    axs[1,1].plot(time[:-1], dxvals, marker='.', linestyle="-", color="red", markersize=0.1)
    axs[1,1].plot(time[:-1], [min_vel_xval]*(len(time)-1), marker='.', color="blue", markersize=0.1)
    axs[1,1].plot(time[:-1], [max_vel_xval]*(len(time)-1), marker='.', color="blue", markersize=0.1)
    axs[1,1].set_ylabel('velocity of diamond center')
    axs[1,1].grid(True)

    axs[2,1].plot(time[:-2], ddxvals, marker='.', linestyle="-", color="red", markersize=0.1)
    axs[2,1].plot(time[:-2], [min_acc_xval]*(len(time)-2), marker='.', linestyle="-", color="blue", markersize=0.1)
    axs[2,1].plot(time[:-2], [max_acc_xval]*(len(time)-2), marker='.', linestyle="-", color="blue", markersize=0.1)
    axs[2,1].set_ylabel('acceleration of diamond center')
    axs[2,1].set_ylim([-20, 20])
    axs[2,1].set_xlabel('time')
    axs[2,1].grid(True)

    axs[0,2].plot(time, dvals, marker='.', linestyle="-", color="red", markersize=0.1)
    axs[0,2].plot(time, [min_dval]*(len(time)), marker='.', color="blue", markersize=0.1)
    # axs[6].plot(time[j][q], [max_dval]*(len(time[j][q])), marker='.', color="blue", markersize=0.1)
    axs[0,2].set_ylabel('ordinate of disk center')
    axs[0,2].set_title('DISK CENTER')
    axs[0,2].grid(True)

    axs[1,2].plot(time[:-1], ddvals, marker='.', color="red", markersize=0.1)
    axs[1,2].plot(time[:-1], [min_vel_dval]*(len(time)-1), marker='.', color="blue", markersize=0.1)
    axs[1,2].plot(time[:-1], [max_vel_dval]*(len(time)-1), marker='.', color="blue", markersize=0.1)
    axs[1,2].set_ylabel('velocity of disk center')
    axs[1,2].grid(True)

    axs[2,2].plot(time[:-2], dddvals, marker='.', color="red", markersize=0.1)
    axs[2,2].plot(time[:-2], [min_acc_dval]*(len(time)-2), marker='.', color="blue", markersize=0.1)
    axs[2,2].plot(time[:-2], [max_acc_dval]*(len(time)-2), marker='.', color="blue", markersize=0.1)
    axs[2,2].set_xlabel('time')
    axs[2,2].set_ylabel('acceleration of disk center')
    # axs[8].set_ylim([-2000, 2000])
    axs[2,2].grid(True)
    
    fig.savefig(path + str(j) + '-' + str(q) + '.png')
    plt.close(fig)

def VisualizeTrace(rangles, xvals, dvals, j, q, r, 
                   current_rough_pts, pear, blpear, angles, n, original_disk, path, framerate):
    local_path = path+'pics'+str(j)+'-'+str(q)
    if os.path.exists(local_path) == False: os.mkdir(local_path)
    else: shutil.rmtree(local_path); os.mkdir(local_path)
    min_xlimit, max_xlimit = np.min(xvals)-5, np.max(xvals)+5
    min_ylimit, max_ylimit = pear.bounds[3]+1, -pear.bounds[3]-1
    count = 0
    for k in tqdm(range(1, len(xvals), framerate)):
        angle, xval, xstep = rangles[k]*pi/180, xvals[k], dvals[k]
        mod_pear = transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(pear, -angle, origin=Point(0.0,0.0), use_radians=True))
        plot_line(mod_pear, color=(1,0,0), add_points=False, label="target")
        # plot_line(LineString([(mod_pear.bounds[2], -20), (mod_pear.bounds[2], 20)]), color=(1,0,0), add_points=False, alpha=0.7)
        plot_line(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(blpear, -angle, origin=Point(0.0,0.0), use_radians=True)), 
                color=(0.5,0.2,0),  add_points=False, label="balooned")
        # plot_line(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(rough, -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #           color=(0,0,1),  add_points=False, label="rough")
        plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(Point(0,0), -angle, origin=Point(0.0,0.0), use_radians=True)), 
                color=(1,0,0), label="center")
        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(LineString(bruting_traces[j][k]), -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(0,1,0), markersize=1, label="bruting trace")
        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(LineString(disk_traces[j][k]), -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(1,0,1), markersize=1, label="disk trace")
        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(bruting_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(0,1,0), markersize=5, label="touch point")
        # plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(disk_traces[j][k][l], -angle, origin=Point(0.0,0.0), use_radians=True)), 
        #             color=(1,0,1), markersize=5, label="disk center")
        disk = transform(lambda x, y, z=None: (x+r-xstep, y), original_disk)
        plot_polygon(disk, add_points=False, color=(0,0,0), alpha=0.4)
        disk = rotate(transform(lambda x, y, z=None: (x-xval, y), disk), angle, origin=Point(0.0,0.0), use_radians=True)
        current_rough_pts = GetCurrentShape(current_rough_pts, disk, angles, n)
        plot_points(transform(lambda x, y, z=None: (x+(xval), y), affinity.rotate(MultiPoint(current_rough_pts), -angle, origin=Point(0.0,0.0), use_radians=True)), 
                    color=(0,0,1), markersize=1, label="rough")
        plt.rcParams["figure.figsize"] = (8.0, 8.0)
        plt.grid(True); plt.legend()
        plt.xlim(min_xlimit, max_xlimit); plt.ylim(min_ylimit, max_ylimit)
        plt.savefig(local_path + '/' + str(count) + ".png"); plt.close()
        count += 1
    with imageio.get_writer(path+'/movie'+str(j)+'-'+str(q)+'.gif', mode='I', duration=0) as writer:
        for l in range(count):
            writer.append_data(imageio.v2.imread(local_path+'/'+str(l)+".png"))

def DecreaseDelta(prev_delta, cur_delta):
    return (prev_delta+cur_delta)/2

def cut(line, distance):
    if distance <= 0.0 or distance >= line.length:
        return [LineString(line)]
    coords = list(line.coords)
    for i, p in enumerate(coords):
        pd = line.project(Point(p))
        if pd == distance:
            return [
                LineString(coords[:i+1]),
                LineString(coords[i:])]
        if pd > distance:
            cp = line.interpolate(distance)
            return [
                LineString(coords[:i] + [(cp.x, cp.y)]),
                LineString([(cp.x, cp.y)] + coords[i:])]

# Algorithm

In [95]:
with open('input.info', 'r') as f: input_data = [line.split(' ') for line in f.readlines()]
R = float(input_data[0][0]); n = int(input_data[0][1]); contain_percent = float(input_data[0][2])
min_vel_rangle = float(input_data[1][0]); min_acc_rangle = float(input_data[1][1])
max_vel_rangle = float(input_data[1][2]); max_acc_rangle = float(input_data[1][3])
min_xval = float(input_data[2][0]); min_vel_xval = float(input_data[2][1]); min_acc_xval = float(input_data[2][2])
max_xval = float(input_data[2][3]); max_vel_xval = float(input_data[2][4]); max_acc_xval = float(input_data[2][5])
min_dval = float(input_data[3][0]); min_vel_dval = float(input_data[3][1]); min_acc_dval = float(input_data[3][2])
max_dval = float(input_data[3][3]); max_vel_dval = float(input_data[3][4]); max_acc_dval = float(input_data[3][5])
max_delta_distance = float(input_data[4][0]); cycle_count = int(input_data[4][1])
rough_filename = input_data[5][0]; diamond_filename = input_data[5][1]; output = input_data[5][2]
r, disk_delta_translation, path, framerate = R+max_xval, max_delta_distance/cycle_count, 'algo_main/', 20
bruting_time = 26.0

rough, pear, angles = GetContourFromOBJ(rough_filename), GetSmoothedPear(GetContourFromOBJ(diamond_filename)), [2*pi*i/n for i in range(n)]
blpear, val = GetBaloonedContourWithContainPercentage(rough, pear, n, contain_percent)
spts, intersection_angles = GetOrderedIntersectionPoints(rough, blpear)
indices, N = GetReperIndices(angles, intersection_angles)
rough_pts, blpear_pts = Contour2Points(rough, 0, n), Contour2Points(blpear, 0, n)

cycle_traces, sptss = [], []
for j in range(0, N, 2):
    local_cycle_traces, local_sptss, i_min, i_max = [], [], indices[j], indices[j+1]
    cycle_rough, cycle_ground = LineString(rough_pts[i_min:i_max]), LineString(blpear_pts[(i_min-50):(i_max+50)])
    local_cycle_traces.append(cycle_ground); local_sptss.append([blpear_pts[i_min], blpear_pts[i_max-1]])
    dst_max = GetMaxDistance(rough_pts[i_min:i_max], LineString(blpear_pts[i_min:i_max]))
    local_cycle_count = floor(dst_max / max_delta_distance)+1
    cycle_trace, spts, dst = GetNextCycleContour(cycle_rough, cycle_ground, max_delta_distance, disk_delta_translation)
    local_cycle_traces.append(cycle_trace); local_cycle_traces.append(cycle_trace); local_sptss.append(spts); local_sptss.append(spts)
    for k in range(local_cycle_count):
        cycle_trace, spts, dst = GetNextCycleContour(cycle_rough, local_cycle_traces[-1], max_delta_distance, disk_delta_translation)
        local_cycle_traces.append(cycle_trace); local_cycle_traces.append(cycle_trace); local_sptss.append(spts); local_sptss.append(spts)
        if len(spts) == 0: break
    local_cycle_traces.pop(); local_sptss.pop()
    cycle_traces.append(local_cycle_traces); sptss.append(local_sptss)

cut_cycle_traces = []
for j in range(len(cycle_traces)):
    local_cut_cycle_traces = []
    for k in range(0, len(cycle_traces[j]), 2):
        current_trace1, current_trace2, cut_pts = cycle_traces[j][k], cycle_traces[j][k+1], sptss[j][k]
        current_trace1_pts = np.array([Point(pt[0], pt[1]) for pt in current_trace1.coords])
        current_trace2_pts = np.array([Point(pt[0], pt[1]) for pt in current_trace2.coords])
        cut_angle1, cut_angle2 = matan(cut_pts[0]), matan(cut_pts[-1])
        angles1, angles2 = np.array([matan(pt) for pt in current_trace1_pts]), np.array([matan(pt) for pt in current_trace2_pts])
        idx1, idx2 = np.where((cut_angle1 <= angles1)*(angles1 <= cut_angle2))[0], np.where((angles2 >= cut_angle1)*(angles2 <= cut_angle2))[0]
        trace1, trace2, maxlen = LineString(current_trace1_pts[idx1].tolist()),  LineString(current_trace2_pts[idx2].tolist()), max(idx1.size,idx2.size)
        local_angles = [(1-alpha)*cut_angle1+alpha*cut_angle2 for alpha in [i/maxlen for i in range(maxlen)]]
        trace1_pts, trace2_pts = [], []
        for angle in local_angles:
            line = LineString([(0,0), (30*cos(angle), 30*sin(angle))])
            pt1, pt2 = trace1.intersection(line), trace2.intersection(line)
            if pt1.geom_type != 'Point' or pt2.geom_type != 'Point': continue
            trace1_pts.append(pt1); trace2_pts.append(pt2)
        local_cut_cycle_traces.append(trace1_pts); local_cut_cycle_traces.append(trace2_pts)
    cut_cycle_traces.append(local_cut_cycle_traces)

bruting_cycle_traces = []
for j in range(len(cut_cycle_traces)):
    bruting_cycle_trace = []
    for k in range(0, len(cut_cycle_traces[j]), 2):
        bruting_cycle_trace.append(cut_cycle_traces[j][k])
        for i in range(cycle_count-1): 
            bruting_cycle_trace.append(GetNextBrutingCycle(cut_cycle_traces[j][k], cut_cycle_traces[j][k+1], (i+1)*disk_delta_translation))
        bruting_cycle_trace.append(cut_cycle_traces[j][k+1])
    bruting_cycle_traces.append(bruting_cycle_trace)

bruting_traces, disk_traces = [], []
for j in range(len(bruting_cycle_traces)):
    local_bruting_traces, local_disk_traces = [], []
    for k in range(0, len(bruting_cycle_traces[j]), cycle_count+1):
        for q in range(k, k+cycle_count):
            trace1, trace2 = bruting_cycle_traces[j][q], bruting_cycle_traces[j][q+1]
            local_local_bruting_trace, local_local_bruting_trace, local_len = [], [], len(bruting_cycle_traces[j][q])
            local_bruting_traces.append(trace1); 
            local_disk_traces.append([Point(pt[0], pt[1]) for pt in LineString(trace1).offset_curve(-R, join_style=2).coords])
            for l in range(local_len): 
                alpha = l/local_len; 
                local_local_bruting_trace.append(Point((1-alpha)*trace1[l].x+alpha*trace2[l].x, (1-alpha)*trace1[l].y+alpha*trace2[l].y))
            local_local_bruting_trace.reverse(); 
            local_bruting_traces.append(local_local_bruting_trace); 
            local_disk_traces.append([Point(pt[0], pt[1]) for pt in LineString(local_local_bruting_trace).offset_curve(R, join_style=2).coords])
    local_bruting_traces.reverse(); 
    local_disk_traces.reverse()
    bruting_traces.append(local_bruting_traces)
    disk_traces.append(local_disk_traces)

concatenated_bruting_traces, concatenated_disk_traces = [], []          
for j in range(len(bruting_traces)):
    local_concatenated_bruting_traces, local_concatenated_disk_traces = [], []
    for q in range(int(len(cut_cycle_traces[j])/2)):
        local_local_concatenated_bruting_traces, local_local_concatenated_disk_traces = [], []
        for k in range(q*2*cycle_count, q*2*cycle_count+2):
            bruting_trace1, bruting_trace2 = bruting_traces[j][k], bruting_traces[j][k+1]
            disk_trace1, disk_trace2 = disk_traces[j][k], disk_traces[j][k+1]
            for pt in bruting_trace1: local_local_concatenated_bruting_traces.append(pt)
            for pt in disk_trace1: local_local_concatenated_disk_traces.append(pt)
            pt0, pt1, pt2 = bruting_trace1[-1], disk_trace1[-1], disk_trace2[0]
            angle = acos(abs(pt1.x*pt2.x+pt1.y*pt2.y)/(sqrt(pt1.x**2+pt1.y**2)*sqrt(pt2.x**2+pt2.y**2)))
            num_pts = floor((angle/(2*pi))*n)
            angle1, angle2 = atan2(pt1.y-pt0.y, pt1.x-pt0.x), atan2(pt2.y-pt0.y, pt2.x-pt0.x)
            anglec, rc = matan(pt0), Point(0,0).distance(pt0)
            concatenate_bruting_trace_pts, concatenate_disk_trace_pts = [], []
            for p in range(num_pts):
                alpha = p / num_pts
                cur_angle = (1-alpha)*angle1+alpha*angle2
                concatenate_bruting_trace_pts.append(pt0)
                concatenate_disk_trace_pts.append(Point(pt0.x+R*cos(cur_angle), pt0.y+R*sin(cur_angle)))
            for pt in concatenate_bruting_trace_pts: local_local_concatenated_bruting_traces.append(pt)
            for pt in concatenate_disk_trace_pts: local_local_concatenated_disk_traces.append(pt)
        for k in range(cycle_count):
            local_concatenated_bruting_traces.append(local_local_concatenated_bruting_traces)
            local_concatenated_disk_traces.append(local_local_concatenated_disk_traces)
    concatenated_bruting_traces.append(local_concatenated_bruting_traces)
    concatenated_disk_traces.append(local_concatenated_disk_traces)

In [ ]:
cconcatenated_disk_traces = []
for j in range(len(concatenated_disk_traces)):
    local_cconcatenated_disk_traces = []
    for q in range(0, len(concatenated_disk_traces[j]), cycle_count):
        local_local_cconcatenated_disk_traces = []
        line = LineString(concatenated_disk_traces[j][q])
        delta_length = line.length/len(concatenated_disk_traces[j][q])
        for l in range(len(concatenated_disk_traces[j][q])-1):
            pt = GetPointOnLineWithDistance(line, delta_length)
            local_local_cconcatenated_disk_traces.append(pt)
            ct = cut(line, delta_length)
            if len(ct) == 1: break
            line = ct[1]
        local_local_cconcatenated_disk_traces.append(concatenated_disk_traces[j][q][-1])
        for k in range(cycle_count):
            local_cconcatenated_disk_traces.append(local_local_cconcatenated_disk_traces)
    cconcatenated_disk_traces.append(local_cconcatenated_disk_traces)
concatenated_disk_traces = cconcatenated_disk_traces

In [114]:
concatenated_bruting_lines = [[LineString(concatenated_bruting_traces[j][q]) for q in range(len(concatenated_bruting_traces[j]))] for j in range(len(concatenated_bruting_traces))]
concatenated_disk_lines = [[LineString(concatenated_disk_traces[j][q]) for q in range(len(concatenated_disk_traces[j]))] for j in range(len(concatenated_disk_traces))]\

result_disk_traces, delta_times = [], []
delta_time = 1e-2
for j in range(len(concatenated_disk_lines)):
    local_result_disk_traces, local_delta_time = [], []
    for q in range(0, len(concatenated_disk_lines[j]), cycle_count):
        
        line = concatenated_disk_lines[j][q]
        line_length = line.length
        global_cur_length = line.length/ min((int(n/2)-100), len(list(line.coords)))

        local_local_result_disk_traces = []
        pt0 = Point(line.coords[0][0], line.coords[0][1])
        angle0 = -atan2(pt0.y, pt0.x)
        xval0 = r-rotate(pt0, angle0, origin=Point(0,0), use_radians=True).x
        local_local_result_disk_traces.append(pt0)

        cur_length = global_cur_length
        while True:
            pt1 = line.interpolate(cur_length, normalized=False)
            angle1 = -atan2(pt1.y, pt1.x)
            angle_velocity = abs(angle1-angle0)/delta_time
            if angle_velocity > max_vel_rangle: 
                cur_length = DecreaseDelta(0, cur_length); 
                continue
            else:
                xval1 = r-rotate(pt1, angle1, origin=Point(0,0), use_radians=True).x
                xval_velocity = abs(xval1-xval0)/delta_time
                if xval_velocity > max_vel_xval: 
                    cur_length = DecreaseDelta(0, cur_length); 
                    continue
                else: 
                    local_local_result_disk_traces.append(pt1)
                    line = cut(line, cur_length)[1]
                    break
        
        while True:
            # print(line.length, end='\r')
            cur_length_angle = cur_length
            while True:
                line_length = line.length
                if abs(line_length) < 1e-14: break
                if len(local_local_result_disk_traces) > int(n/2)-1: break
                pt2 = line.interpolate(cur_length_angle, normalized=False)
                angle2 = -atan2(pt2.y, pt2.x)
                angle_velocity = abs(angle2-angle1)/delta_time
                if angle_velocity > max_vel_rangle: 
                    cur_length_angle = DecreaseDelta(0, cur_length_angle); 
                    continue
                else:
                    angle_acceleration = abs((angle2-angle1)/delta_time-(angle1-angle0)/delta_time)/delta_time
                    if angle_acceleration > max_acc_rangle: 
                        cur_length_angle = DecreaseDelta(0, cur_length_angle); 
                        continue
                    else: break
                        
            cur_length_xval = cur_length_angle
            while True:
                print(cur_length_xval, end='\r')
                line_length = line.length
                if abs(line_length) < 1e-14: break
                if len(local_local_result_disk_traces) > int(n/2)-1: break
                pt2 = line.interpolate(cur_length_xval, normalized=False)
                angle2 = -atan2(pt2.y, pt2.x)
                xval2 = r-rotate(pt2, angle2, origin=Point(0,0), use_radians=True).x
                xval_velocity = abs(xval2-xval1)/delta_time
                if xval_velocity > max_vel_xval: 
                    cur_length_xval = DecreaseDelta(0, cur_length_xval); 
                    continue
                else:
                    xval_acceleration = abs((xval2-xval1)/delta_time-(xval1-xval0)/delta_time)/delta_time
                    if xval_acceleration > max_acc_xval: 
                        cur_length_xval = DecreaseDelta(0, cur_length_xval); 
                        continue
                    else:
                        local_local_result_disk_traces.append(pt2)
                        pt0 = pt1; pt1 = pt2; 
                        xval0 = xval1; xval1 = xval2
                        angle0 = angle1; angle1 = angle2

                        ct = cut(line, cur_length_xval)
                        if len(ct) == 1: break
                        line = cut(line, cur_length_xval)[1]
                        cur_length = min(global_cur_length, line.length-cur_length_xval)
                        continue

            if abs(line_length) < 1e-14: break
            if len(local_local_result_disk_traces) > int(n/2)-1: break

        for k in range(cycle_count): 
            local_result_disk_traces.append(local_local_result_disk_traces); local_delta_time.append(delta_time)
    result_disk_traces.append(local_result_disk_traces); delta_times.append(local_delta_time)

# result_disk_traces = concatenated_disk_traces
# delta_times = []
# for j in range(len(result_disk_traces)):
#     local_delta_time = []
#     for q in range(0, len(result_disk_traces[j]), cycle_count):
#         delta_ttime = 1/len(result_disk_traces[j][q])
#         for k in range(cycle_count):
#             local_delta_time.append(delta_ttime)
#     delta_times.append(local_delta_time)

xvals, rangles, dvals, time, delta = [], [], [], [], []
for j in range(len(result_disk_traces)):
    local_xvals, local_rangles, local_dvals, local_time, local_delta = [], [], [], [], []
    for q in range(0, len(result_disk_traces[j]), cycle_count):
        local_local_xvals, local_local_rangles, local_local_dvals, local_local_time, local_local_delta = [], [], [], [], []
        whole_length, cur_length, xstep = 0, 0, 0
        for k in range(cycle_count): whole_length += len(result_disk_traces[j][q+k])
        whole_length -= (cycle_count-1)
        ltime, delta_ttime, delta_xstep = 0, delta_times[j][q], 1/whole_length
        for k in range(cycle_count):
            if k > 0:
                angle = atan2(result_disk_traces[j][q+k][0].y, result_disk_traces[j][q+k][0].x)
                xval = r-affinity.rotate(result_disk_traces[j][q+k][0], -angle, origin=Point(0.0,0.0), use_radians=True).x
                dlt, km = local_local_xvals[-1]-xval, 1
            else:
                dlt, km = 0, 0
            for l in range(km, len(result_disk_traces[j][q+k])):
                angle = atan2(result_disk_traces[j][q+k][l].y, result_disk_traces[j][q+k][l].x)
                xval = r-affinity.rotate(result_disk_traces[j][q+k][l], -angle, origin=Point(0.0,0.0), use_radians=True).x +dlt
                local_local_xvals.append(xval); local_local_rangles.append(angle*180/pi)
                # local_local_time.append(bruting_time*cur_length/whole_length)
                local_local_time.append(ltime)
                ltime += delta_ttime
            local_local_delta.append(dlt)
        for k in range(cycle_count):
            km = 1 if k > 0 else 0
            for l in range(km, len(result_disk_traces[j][q+k])):
                xstep += delta_xstep * (local_local_delta[1]*1)
                local_local_dvals.append(cycle_count*xstep); 
        for m in range(len(local_local_xvals)):
            local_local_xvals[m] -= local_local_dvals[m]   
        local_xvals.append(local_local_xvals); local_rangles.append(local_local_rangles); 
        local_dvals.append(local_local_dvals); local_time.append(local_local_time); 
        local_delta.append(local_local_delta)
    xvals.append(local_xvals); rangles.append(local_rangles); dvals.append(local_dvals); time.append(local_time)
    delta.append(local_delta)

if os.path.exists(output): os.remove(output)
for j in range(len(rangles)):
    for q in range(len(rangles[j])):
        with open(output, 'a') as f:
            f.write('; sector\ncycle\n{\n')
            f.write('\tRx_0 ' + str(round(rangles[j][q][0], 6)) + '\n')
            f.write('\tYs_0 ' + str(round(xvals[j][q][0], 6)) + '\n')
            f.write('\tYf_0 ' + str(round(dvals[j][q][0], 6)) + '\n')
            f.write('\tfeed\n\t{\n\t\t' + str(cycle_count) + ' ' + str(round(dvals[j][q][-1], 6)) + '\n\t}\n')
            f.write('\tT_step ' + str(round(delta_times[j][q], 6)) + '\n\tshape\n\t{\n')
            for k in range(1, len(xvals[j][q])):
                f.write('\t\t' + str(round(rangles[j][q][k], 6)) + ' ' + str(round(xvals[j][q][k], 6)) + '\n')
            f.write('\t}\n}\n\n')


KeyboardInterrupt: 

In [ ]:
concatenated_bruting_lines = [[LineString(concatenated_bruting_traces[j][q]) for q in range(len(concatenated_bruting_traces[j]))] for j in range(len(concatenated_bruting_traces))]
concatenated_disk_lines = [[LineString(concatenated_disk_traces[j][q]) for q in range(len(concatenated_disk_traces[j]))] for j in range(len(concatenated_disk_traces))]

result_disk_traces = []
for j in range(len(concatenated_disk_lines)):
    local_result_disk_traces = []
    for q in range(0, len(concatenated_disk_lines[j]), cycle_count):
# for j in range(1):
#     local_result_disk_traces = []
#     for q in range(0, 3, 2):
        delta_time = 1e-1
        line = concatenated_disk_lines[j][q]
        global_cur_length = line.length/(len(list(line.coords))*2/3)
        local_local_result_disk_traces = []
        pt0 = Point(line.coords[0][0], line.coords[0][1])
        angle0 = atan2(pt0.y, pt0.x)
        xval0 = rotate(pt0, -angle0, origin=Point(0,0), use_radians=True).x
        local_local_result_disk_traces.append(pt0)
        cur_length = global_cur_length
        while True:
            pt1 = line.interpolate(cur_length, normalized=False)
            angle1 = atan2(pt1.y, pt1.x)
            if abs(angle1-angle0)/delta_time > max_vel_rangle: cur_length = DecreaseDelta(cur_length); continue
            else:
                xval1 = rotate(pt1, -angle1, origin=Point(0,0), use_radians=True).x
                if abs(xval1-xval0)/delta_time > max_vel_xval: cur_length = DecreaseDelta(cur_length); continue
                else: local_local_result_disk_traces.append(pt1); break
        cur_length, step = line.project(pt1)+global_cur_length, 2
        while True:
            if cur_length > line.length: break
            # print(cur_length, end='\r')
            pt2 = line.interpolate(cur_length, normalized=False)
            angle2 = atan2(pt2.y, pt2.x)
            if abs(angle2-angle1)/delta_time > max_vel_rangle: cur_length = DecreaseDelta(cur_length); continue
            else:
                if abs((angle2-angle1)/delta_time-(angle1-angle0)/delta_time) > max_acc_rangle: cur_length = DecreaseDelta(cur_length); continue
                else:
                    xval2 = rotate(pt2, -angle2, origin=Point(0,0), use_radians=True).x
                    if abs(xval2-xval1)/delta_time > max_vel_xval: cur_length = DecreaseDelta(cur_length); continue
                    else:
                        if abs((xval2-xval1)/delta_time-(xval1-xval0)/delta_time) > max_acc_xval: cur_length = DecreaseDelta(cur_length); continue
                        else:
                            local_local_result_disk_traces.append(pt2); 
                            pt0 = pt1; pt1 = pt2; 
                            # cur_length = global_cur_length*step
                            cur_length = line.project(pt1)+global_cur_length
                            step += 1
                            print(step, len(local_local_result_disk_traces), cur_length, line.length, end="\r")
                            continue
        # print(local_local_result_disk_traces[-1].distance(Point(line.coords[-1][0], line.coords[-1][1])))
        # print(len(local_local_result_disk_traces))
        for k in range(cycle_count): local_result_disk_traces.append(local_local_result_disk_traces)
    result_disk_traces.append(local_result_disk_traces)

xvals, rangles, dvals, time, delta, delta_time = [], [], [], [], [], []
for j in range(len(result_disk_traces)):
    local_xvals, local_rangles, local_dvals, local_time, local_delta, local_delta_time = [], [], [], [], [], []
    for q in range(0, len(result_disk_traces[j]), cycle_count):
        local_local_xvals, local_local_rangles, local_local_dvals, local_local_time, local_local_delta = [], [], [], [], []
        whole_length, cur_length, xstep = 0, 0, 0
        for k in range(cycle_count): whole_length += len(result_disk_traces[j][q+k])
        whole_length -= (cycle_count-1)
        delta_ttime, delta_xstep = bruting_time/whole_length, 1/whole_length
        for k in range(cycle_count):
            if k > 0:
                angle = atan2(result_disk_traces[j][q+k][0].y, result_disk_traces[j][q+k][0].x)
                xval = r-affinity.rotate(result_disk_traces[j][q+k][0], -angle, origin=Point(0.0,0.0), use_radians=True).x
                dlt, km = local_local_xvals[-1]-xval, 1
            else:
                dlt, km = 0, 0
            for l in range(km, len(result_disk_traces[j][q+k])):
                angle = atan2(result_disk_traces[j][q+k][l].y, result_disk_traces[j][q+k][l].x)
                xval = r-affinity.rotate(result_disk_traces[j][q+k][l], -angle, origin=Point(0.0,0.0), use_radians=True).x +dlt
                local_local_xvals.append(xval); local_local_rangles.append(angle*180/pi)
                local_local_time.append(bruting_time*cur_length/whole_length)
                cur_length += 1
            local_local_delta.append(dlt)
        for k in range(cycle_count):
            km = 1 if k > 0 else 0
            for l in range(km, len(result_disk_traces[j][q+k])):
                xstep += delta_xstep * (local_local_delta[1]*1)
                local_local_dvals.append(cycle_count*xstep); 
        for m in range(len(local_local_xvals)):
            local_local_xvals[m] -= local_local_dvals[m]   
        local_xvals.append(local_local_xvals); local_rangles.append(local_local_rangles); 
        local_dvals.append(local_local_dvals); local_time.append(local_local_time); 
        local_delta.append(local_local_delta); local_delta_time.append(delta_ttime)
    xvals.append(local_xvals); rangles.append(local_rangles); dvals.append(local_dvals); time.append(local_time)
    delta.append(local_delta); delta_time.append(local_delta_time)

In [60]:
# print(len(result_disk_traces))
# print(len(result_disk_traces[0]))
# print(len(result_disk_traces[0][0]))

xvals, rangles, dvals, time, delta = [], [], [], [], []
j, q = 0, 3
cur_length = 0
bruting_time = delta_time * len(result_disk_traces[j][q])
for l in range(len(result_disk_traces[j][q])):
      angle = atan2(result_disk_traces[j][q][l].y, result_disk_traces[j][q][l].x)
      xval = r-affinity.rotate(result_disk_traces[j][q][l], -angle, origin=Point(0.0,0.0), use_radians=True).x +dlt
      xvals.append(xval); rangles.append(angle*180/pi)
      time.append(bruting_time*cur_length/whole_length)
      cur_length += 1

headers = ['cycle','amount of points','rotation velocity','rotation acceleration','diamond','diamond velocity','diamond acceleration','disk','disk velocity','disk acceleration']
table = TestTrace(time, rangles, xvals, dvals, j, q, False, n,
              min_vel_rangle, min_acc_rangle, max_vel_rangle, max_acc_rangle,
              min_xval, min_vel_xval, min_acc_xval, max_xval, max_vel_xval, max_acc_xval,
              min_dval, min_vel_dval, min_acc_dval, max_dval, max_vel_dval, max_acc_dval)
print(Fore.GREEN+'Trace in limits'+Fore.RESET 
      if np.where(np.array(table[2:])!='\x1b[1m\x1b[42msuccess\x1b[49m\x1b[22m (0)')[0].size==0 
      else Fore.RED+'Trace not in limits'+Fore.RESET)
# print(tabulate(table, headers))

Trace in limits


In [ ]:
# for j in range(len(concatenated_bruting_lines)):
#     for q in range(len(concatenated_bruting_lines[j])):
#         plot_line(concatenated_bruting_lines[j][q], add_points=False, color=(0,1,0))
#         plot_line(concatenated_disk_lines[j][q], add_points=False, color=(1,0,1))

for j in range(len(result_disk_traces)):
    for q in range(len(result_disk_traces[j])):
        # plot_line(concatenated_bruting_lines[j][q], add_points=False, color=(0,1,0))
        plot_line(LineString(result_disk_traces[j][q]), add_points=False, color=(1,0,1))

# for j in range(len(cconcatenated_disk_traces)):
#     for q in range(len(cconcatenated_disk_traces[j])):
#         plot_line(LineString(cconcatenated_disk_traces[j][q]), add_points=False, color=(1,0,1))

# Testing

In [110]:
headers = ['cycle','amount of points','rotation velocity','rotation acceleration','diamond','diamond velocity','diamond acceleration','disk','disk velocity','disk acceleration']
table = [TestTrace(time[j][q], rangles[j][q], xvals[j][q], dvals[j][q], j, q, False, n,
              min_vel_rangle, min_acc_rangle, max_vel_rangle, max_acc_rangle,
              min_xval, min_vel_xval, min_acc_xval, max_xval, max_vel_xval, max_acc_xval,
              min_dval, min_vel_dval, min_acc_dval, max_dval, max_vel_dval, max_acc_dval) for j,q in 
              [item for sublists in [list(it) for it in [itertools.product([k],range(len(xvals[k]))) for k in range(len(xvals))]] for item in sublists]]

bad_flag = 0
for i in range(9, len([item for sublists in [t[2] for t in table] for item in sublists]), 30):
      if [item for sublists in [t[2] for t in table] for item in sublists][i]!='s': bad_flag = 1; break 
if np.where(np.array([item for sublists in [t[2:] for t in table] for item in sublists])!='\x1b[1m\x1b[42msuccess\x1b[49m\x1b[22m (0)')[0].size==0 and bad_flag==0:
      print(Fore.GREEN+'Trace in limits'+Fore.RESET)
else: print(Fore.RED+'Trace not in limits'+Fore.RESET)
print(tabulate(table, headers, tablefmt='fancy_grid'))

Trace in limits
╒═════════╤════════════════════╤═════════════════════╤═════════════════════════╤═════════════╤════════════════════╤════════════════════════╤═════════════╤═════════════════╤═════════════════════╕
│ cycle   │ amount of points   │ rotation velocity   │ rotation acceleration   │ diamond     │ diamond velocity   │ diamond acceleration   │ disk        │ disk velocity   │ disk acceleration   │
╞═════════╪════════════════════╪═════════════════════╪═════════════════════════╪═════════════╪════════════════════╪════════════════════════╪═════════════╪═════════════════╪═════════════════════╡
│ 1-1     │ success (757)      │ success (0)         │ success (0)             │ success (0) │ success (0)        │ success (0)            │ success (0) │ success (0)     │ success (0)         │
├─────────┼────────────────────┼─────────────────────┼─────────────────────────┼─────────────┼────────────────────┼────────────────────────┼─────────────┼─────────────────┼─────────────────────┤
│ 1-2    

In [111]:
for j in range(len(xvals)):
    for q in range(len(xvals[j])):
        GraphTrace(time[j][q], rangles[j][q], xvals[j][q], dvals[j][q], R, path, j, q,
              min_vel_rangle, min_acc_rangle, max_vel_rangle, max_acc_rangle,
              min_xval, min_vel_xval, min_acc_xval, max_xval, max_vel_xval, max_acc_xval,
              min_dval, min_vel_dval, min_acc_dval, max_dval, max_vel_dval, max_acc_dval)

In [ ]:
original_disk, current_rough_pts = Point(0,0).buffer(R, quad_segs=n), rough_pts
for j in tqdm(range(len(xvals))):
    for q in tqdm(range(len(xvals[j]))):
        VisualizeTrace(rangles[j][q], xvals[j][q], dvals[j][q], j, q, r, 
                   current_rough_pts, pear, blpear, angles, n, original_disk, path, framerate)